In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay,precision_score,recall_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_validate

data = pd.read_csv('/content/HeartDiesease.csv')
data.head(5)

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [5]:
Y=data["target"]
X=data.drop("target",axis=1)

In [8]:
sc = StandardScaler()
X=sc.fit_transform(X)

array([[-1.46672783,  0.55599543, -1.31835093, ..., -0.79521891,
        -0.84979236, -1.02321701],
       [-0.50460037, -1.79857595, -0.24893198, ..., -0.79521891,
         0.07111913,  0.61558278],
       [-1.78743698,  0.55599543, -1.31835093, ..., -0.79521891,
        -0.84979236, -1.02321701],
       ...,
       [ 0.35062404,  0.55599543,  0.82048698, ...,  1.25751537,
         0.25530143,  0.61558278],
       [ 0.35062404, -1.79857595, -1.31835093, ..., -0.79521891,
        -0.84979236,  0.61558278],
       [-1.68053393,  0.55599543, -0.24893198, ..., -0.79521891,
        -0.84979236, -1.02321701]])

In [52]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [53]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((892, 11), (298, 11), (892,), (298,))

### Logistic Regression

In [80]:
clf = LogisticRegression(n_jobs=-1,C=0.1)
clf.fit(x_train,y_train)
cvalidate = cross_validate(clf,X,Y,cv=5,scoring="accuracy",return_train_score=True)
print("test_score",cvalidate["test_score"].mean())
print("train_score",cvalidate["train_score"].mean())
# cvalidate

test_score 0.8092436974789916
train_score 0.8296218487394957


## SVM

In [81]:
clf = SVC()
clf.fit(x_train,y_train)
cvalidate = cross_validate(clf,X,Y,cv=5,scoring="accuracy",return_train_score=True)
print("test_score",cvalidate["test_score"].mean())
print("train_score",cvalidate["train_score"].mean())

test_score 0.6941176470588235
train_score 0.7176470588235294


## RF

In [83]:
clf = RandomForestClassifier(criterion="log_loss")
clf.fit(x_train,y_train)
cvalidate = cross_validate(clf,X,Y,cv=5,scoring="accuracy",return_train_score=True)
print("test_score",cvalidate["test_score"].mean())
print("train_score",cvalidate["train_score"].mean())

test_score 0.919327731092437
train_score 1.0
